In [1]:
from astropy.io import fits, ascii
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''
catId
  A small integer specifying the source of the objId.  Currently the following catIds are defined.

    catId |  description
   -------|--------------------------------------------------
        0 | Simulated catalog
        1 | Gaia Data Release 1
        2 | Gaia Data Release 2
        3 | Gaia Early Data Release 3
        4 | Gaia Data Release 3
        5 | HSC-SSP Public Data Release 1 (Wide)
        6 | HSC-SSP Public Data Release 1 (Deep+UltraDeep)
        7 | HSC-SSP Public Data Release 2 (Wide)
        8 | HSC-SSP Public Data Release 2 (Deep+UltraDeep)
        9 | HSC-SSP Public Data Release 3 (Wide)
       10 | HSC-SSP Public Data Release 3 (Deep+UltraDeep)
       11 | HSC-SSP Public Data Release 4 (Wide)
       12 | HSC-SSP Public Data Release 4 (Deep+UltraDeep)
     1001 | Sky positions from S21A HSC-SSP (Wide)
     1002 | Sky positions from PS1
     1003 | Sky positions from Gaia
     1004 | Sky positions for regions without PS1 data
    90001 | Messier 15 stars for the engineering observation
    90002 | NGC1904 for the engineering run in November 2022
    90003 | NGC2419 for the engineering run in November 2022
    90004 | NGC5272 for the engineering run in February 2023
    90005 | NGC5904 for the engineering run in February 2023
    90006 | NGC7078 for the engineering run in July 2023
    90007 | NGC7089 for the engineering run in July 2023
    90008 | NGC7099 for the engineering run in July 2023

Comment: The combination of catId  and obj_id must be unique within one design.

Target uploader requirement:
https://pfs-etc.naoj.hawaii.edu/uploader/doc/inputs.html

ob_code, obj_id, obj_type, ra, dec, pmra, pmdec, epoch, exptime, priority, resolution, reference_arm, g_ps1, g_ps1_error
'''

In [2]:
# TODO: make this function useful for one/multiple pointings, plot the cosmology, star, sky and ancillary targets using different colors
def plot_tgt_done(outfn_list, figname, figsize=(8, 8), plot_diffcolor=True):
    '''
    plot the targets that have been assigned to the fibers
    '''
    for i, fn_i in enumerate(outfn_list):
        tgt_id_done, tgt_ra_done, tgt_dec_done = np.loadtxt(fn_i, usecols=(0, 4, 5), unpack=True, dtype='str')
        tgt_ra_done, tgt_dec_done = tgt_ra_done.astype('float'), tgt_dec_done.astype('float')

        mask_cos = [tgt_id_done[i][1:4] == 'Cos' for i in range(len(tgt_id_done))]
        mask_star = [tgt_id_done[i][1:4] == 'Sta' for i in range(len(tgt_id_done))]
        mask_sky = [tgt_id_done[i][1:4] == 'Sky' for i in range(len(tgt_id_done))]
        mask_anc = [tgt_id_done[i][1:4] == 'Anc' for i in range(len(tgt_id_done))]

        if(plot_diffcolor):
            if(i==0): 
                plt.figure(figsize = figsize)
                plt.plot(tgt_ra_done[mask_cos], tgt_dec_done[mask_cos], 'k.', ms=0.5, alpha=0.5, label='cosmology')
                plt.plot(tgt_ra_done[mask_star], tgt_dec_done[mask_star], 'r*', ms=1.5, alpha=1., label='star')
                plt.plot(tgt_ra_done[mask_sky], tgt_dec_done[mask_sky], 'b^', ms=1.5, alpha=1., label='sky')
                plt.plot(tgt_ra_done[mask_anc], tgt_dec_done[mask_anc], 'gs', ms=1.5, alpha=1., label='ancillary')
            else:
                plt.plot(tgt_ra_done[mask_cos], tgt_dec_done[mask_cos], 'k.', ms=0.5, alpha=0.5)
                plt.plot(tgt_ra_done[mask_star], tgt_dec_done[mask_star], 'r*', ms=1.5, alpha=1.)
                plt.plot(tgt_ra_done[mask_sky], tgt_dec_done[mask_sky], 'b^', ms=1.5, alpha=1.)
                plt.plot(tgt_ra_done[mask_anc], tgt_dec_done[mask_anc], 'gs', ms=1.5, alpha=1.)
        else:
            if(i==0): 
                plt.figure(figsize = figsize)
                plt.plot(tgt_ra_done, tgt_dec_done, 'k.', ms=0.5, alpha=0.5, label='targets done')
            else:
                plt.plot(tgt_ra_done, tgt_dec_done, 'k.', ms=0.5, alpha=0.5)

    plt.legend(loc='upper right', fontsize=15, frameon=True)
    plt.xlabel('RA', fontsize=15)
    plt.ylabel('DEC', fontsize=15)
    plt.savefig(figname, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

# get the target list from netflow output and original cosmology target list
+ for each figure configuration
+ star catalog - flux is in nJy? or Jy?
+ sky catalog - flux is missing

In [5]:
fdir_raw_Jan25 = '../data_raw/Jan25/'
prefix_org = 'cmx_01222025_DESI-ELG_'
outdir_Jan25 = '../output/Jan25/' 
fdir_proc_Jan25 = '../data_proc/Jan25/'
prefix = ''

# HST(UTC-10) Jan 22 20:00 for field a and b, same for both visits
otime_list = ['2025-01-23T06:00:00Z', '2025-01-23T06:00:00Z']

#for field, Li_list, otime in zip(['a', 'b'], [['L1', 'L3', 'L4'], ['L1', 'L2', 'L3']], otime_list):
for field, Li_list, otime in zip(['a', ], [['L1', 'L3', 'L4'], ], ['2025-01-23T06:00:00Z', ]):
    infn_list = []
    # read original cosmology targets
    data_cos = ascii.read(fdir_raw_Jan25 + prefix_org + 'field-{}_v{}.csv'.format(field, 1))
    # read processed star targets
    data_star = ascii.read(fdir_proc_Jan25 + 'star_field_{}_targets.ecsv'.format(field))
    # read processed sky targets
    data_sky = ascii.read(fdir_proc_Jan25 + 'sky_field_{}_targets.ecsv'.format(field))

    # read netflow output
    for Li in Li_list[:1]:
        for visit in [1, 2][:1]:
            infn = prefix + 'field_%s_%s_visit%d.txt'%(field, Li, visit)
            infn_list.append(outdir_Jan25+infn)
        
        tgt_id_done = np.loadtxt(outdir_Jan25+infn, usecols=(0, ), unpack=True, dtype='str')
        # match the tgt_id_done cosmology with the original cosmology targets
        mask_cos = np.in1d(data_cos['obj_id'], tgt_id_done)
        # match the tgt_id_done star with the processed star targets
        mask_star = np.in1d(data_star['obj_id'], tgt_id_done)
        # match the tgt_id_done sky with the processed sky targets
        mask_sky = np.in1d(data_sky['obj_id'], tgt_id_done)
        print('field %s, Li %s, visit %d, cosmology %d, star %d, sky %d'%(field, Li, visit, np.sum(mask_cos), np.sum(mask_star), np.sum(mask_sky)))

        # create obj_code for the targets
        
        # write all the matched targets to the final file


    figname = outdir_Jan25+prefix+'fiberassign_'+field+'_done.png'
    #plot_tgt_done(infn_list, figname, figsize=(8, 8), plot_diffcolor=True)

In [12]:
tgt_id_done

array(['_Star_2490093051763096192', '_Cos_38558236403591600',
       '_Sky_2841082372', ..., '_Cos_38558116144481460',
       '_Cos_38558116144482646', '_Cos_38558116144502435'], dtype='<U25')

In [13]:
data_cos['obj_id']

36429049316264247
36429049316252412
36429186755218807
36429186755219312
36429186755220301
36434546874405703
36429049316266601
36429053611231014
36429049316265098
36429186755216222
36429186755220500
